In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import time
from kaggle_secrets import UserSecretsClient
import os
import subprocess

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Ned to get latitude and longitude 

In [3]:
original_df = pd.read_csv('/kaggle/input/indian-villages-database/addresses.csv')
yeasterday_df = pd.read_csv('/kaggle/input/indian-villages-database/villages.csv')

# Decide now , from where to start

In [4]:
original_df = original_df[[ 'country','state','district','taluka','village']]
yeasterday_df = yeasterday_df[[ 'country','state','district','taluka','village']]

In [5]:
limit = 40000
yeasterday_collection = len(yeasterday_df)
print(yeasterday_collection)
original_length = len(original_df)
if original_length - yeasterday_collection > limit:
    target_for_today = yeasterday_collection+limit
else:
    target_for_today = original_length - yeasterday_collection
print(target_for_today)

50
100


In [6]:
target_df = original_df[yeasterday_collection:target_for_today]
target_df

,country,state,district,taluka,village
50,India,andaman-&-nicobar-islands,nicobars,great-nicobar,kondul
51,India,andaman-&-nicobar-islands,nicobars,great-nicobar,kopenheat
52,India,andaman-&-nicobar-islands,nicobars,great-nicobar,lanaya
53,India,andaman-&-nicobar-islands,nicobars,great-nicobar,lawful
54,India,andaman-&-nicobar-islands,nicobars,great-nicobar,laxmi-nagar
55,India,andaman-&-nicobar-islands,nicobars,great-nicobar,makhahu-makachua
56,India,andaman-&-nicobar-islands,nicobars,great-nicobar,minlana-minlan
57,India,andaman-&-nicobar-islands,nicobars,great-nicobar,navy-dera
58,India,andaman-&-nicobar-islands,nicobars,great-nicobar,not-yet-named
59,India,andaman-&-nicobar-islands,nicobars,great-nicobar,not-yet-named-a


In [7]:
latlist = []
lonlist = []

for index, row in tqdm(target_df.iterrows()):
    state = row.iloc[1]
    district = row.iloc[2]
    taluka = row.iloc[3]
    village = row.iloc[4]
    url_to_hit = f"https://www.geonames.org/advanced-search.html?q={village}%2C{taluka}%2C{district}%2C{state}&country=IN&featureClass=&continentCode="  
    r = rq.get(url_to_hit,headers = headers)    
    soup = BeautifulSoup(r.text,'html.parser')   
    lat = soup.find('span',{'class':'latitude'})    
    lon = soup.find('span',{'class':'longitude'})   
    if lat is not None and lon is not None:
        latlist.append(lat.text)
        lonlist.append(lon.text)
    else:
        latlist.append(None)
        lonlist.append(None)

50it [00:31,  1.61it/s]


In [8]:
new_df = target_df.copy()
new_df

,country,state,district,taluka,village
50,India,andaman-&-nicobar-islands,nicobars,great-nicobar,kondul
51,India,andaman-&-nicobar-islands,nicobars,great-nicobar,kopenheat
52,India,andaman-&-nicobar-islands,nicobars,great-nicobar,lanaya
53,India,andaman-&-nicobar-islands,nicobars,great-nicobar,lawful
54,India,andaman-&-nicobar-islands,nicobars,great-nicobar,laxmi-nagar
55,India,andaman-&-nicobar-islands,nicobars,great-nicobar,makhahu-makachua
56,India,andaman-&-nicobar-islands,nicobars,great-nicobar,minlana-minlan
57,India,andaman-&-nicobar-islands,nicobars,great-nicobar,navy-dera
58,India,andaman-&-nicobar-islands,nicobars,great-nicobar,not-yet-named
59,India,andaman-&-nicobar-islands,nicobars,great-nicobar,not-yet-named-a


In [9]:
new_df['latitude'] = latlist
new_df['longitude'] = lonlist
new_df

,country,state,district,taluka,village,latitude,longitude
50,India,andaman-&-nicobar-islands,nicobars,great-nicobar,kondul,None,None
51,India,andaman-&-nicobar-islands,nicobars,great-nicobar,kopenheat,None,None
52,India,andaman-&-nicobar-islands,nicobars,great-nicobar,lanaya,None,None
53,India,andaman-&-nicobar-islands,nicobars,great-nicobar,lawful,None,None
54,India,andaman-&-nicobar-islands,nicobars,great-nicobar,laxmi-nagar,None,None
55,India,andaman-&-nicobar-islands,nicobars,great-nicobar,makhahu-makachua,None,None
56,India,andaman-&-nicobar-islands,nicobars,great-nicobar,minlana-minlan,None,None
57,India,andaman-&-nicobar-islands,nicobars,great-nicobar,navy-dera,None,None
58,India,andaman-&-nicobar-islands,nicobars,great-nicobar,not-yet-named,None,None
59,India,andaman-&-nicobar-islands,nicobars,great-nicobar,not-yet-named-a,None,None


# update data

In [10]:
updated_df = pd.concat([yeasterday_df, new_df], ignore_index=True)
updated_df

,country,state,district,taluka,village,latitude,longitude
0,India,andaman-&-nicobar-islands,nicobars,car-nicobar,arong,NaN,NaN
1,India,andaman-&-nicobar-islands,nicobars,car-nicobar,big-lapati,NaN,NaN
2,India,andaman-&-nicobar-islands,nicobars,car-nicobar,chuckchucha,NaN,NaN
3,India,andaman-&-nicobar-islands,nicobars,car-nicobar,iaf-camp,NaN,NaN
4,India,andaman-&-nicobar-islands,nicobars,car-nicobar,kakana,NaN,NaN
...,...,...,...,...,...,...,...
95,India,andaman-&-nicobar-islands,nicobars,nancowry,alpintu-alpintung,None,None
96,India,andaman-&-nicobar-islands,nicobars,nancowry,alreak,None,None
97,India,andaman-&-nicobar-islands,nicobars,nancowry,alsama,None,None
98,India,andaman-&-nicobar-islands,nicobars,nancowry,altaful,None,None


In [11]:
os.mkdir('datasets')
# todays updated dataset
updated_df.to_csv('datasets/villages.csv')
# original_dataset
original_df.to_csv('datasets/addresses.csv')

# Upload 

In [12]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

In [13]:
print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/indian-villages-database"
}
metadata_file_location = '/kaggle/working/datasets/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    json.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


In [14]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/datasets' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        print("Upload complete")
        break
    except Exception as error:
        print(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file villages.csv


100%|██████████| 6.89k/6.89k [00:00<00:00, 10.6kB/s]
  0%|          | 0.00/31.3M [00:00<?, ?B/s]

Upload successful: villages.csv (7KB)
Starting upload for file addresses.csv


100%|██████████| 31.3M/31.3M [00:00<00:00, 34.9MB/s]


Upload successful: addresses.csv (31MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/indian-villages-database
Upload complete
